<a href="https://colab.research.google.com/github/ameliamansfield/EMR_Data_SepsisAnalysis_Python/blob/main/SepsisAnalysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

import os
import pandas as pd
import sqlite3

folder_path = "/content/drive/MyDrive/2025_PatientLevelDataAnalysis/eicu_csvs"

db_path = os.path.join(folder_path, "eicu_rebuilt.sqlite")
conn = sqlite3.connect(db_path)
cursor = conn.cursor()

df = pd.read_csv("/content/drive/MyDrive/2025_PatientLevelDataAnalysis/eicu_csvs/patient.csv.gz")
df.to_sql("patient", conn, if_exists="replace", index=False)

cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
print(cursor.fetchall())

for file in os.listdir(folder_path):
    if file.endswith(".csv.gz"):
        table_name = file.replace(".csv.gz", "")
        file_path = os.path.join(folder_path, file)
        print(f"Loading {table_name}...")

        df = pd.read_csv(file_path)

        if not df.empty:
            df.to_sql(table_name, conn, if_exists="replace", index=False)

cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
tables = [t[0] for t in cursor.fetchall()]
print("Tables in database:", tables)

for table in tables:
    cursor.execute(f"PRAGMA table_info({table});")
    columns = [col[1] for col in cursor.fetchall()]
    print(f"\nTable: {table}\nColumns: {columns}")



Mounted at /content/drive


OperationalError: unable to open database file

In [ ]:
import os
import pandas as pd
import sqlite3

all_files = os.listdir("/content/drive/MyDrive/2025_PatientLevelDataAnalysis/eicu_csvs/")

def pick(files, exact=None, include=None, exclude=None):
    files_l = [f.lower() for f in files]
    mapping = dict(zip(files_l, files))
    if exact and exact.lower() in mapping:
        return mapping[exact.lower()]
    cand = []
    for fl, orig in mapping.items():
        if include and not any(inc in fl for inc in include):
            continue
        if exclude and any(exc in fl for exc in exclude):
            continue
        cand.append(orig)
    return cand[0] if cand else None

file_patient   = pick(all_files, exact='patient.csv.gz', include=['patient'], exclude=['apache'])
file_diagnosis = pick(all_files, exact='diagnosis.csv.gz', include=['diagnosis','admissiondx'])
file_apache_pt = pick(all_files, exact='apachePatientResult.csv.gz', include=['apachepatientresult'])
file_vital_per = pick(all_files, exact='vitalPeriodic.csv.gz', include=['vitalperiodic'])
file_vital_aper= pick(all_files, exact='vitalAperiodic.csv.gz', include=['vitalaperiodic'])
file_lab       = pick(all_files, exact='lab.csv.gz', include=['lab'], exclude=['microlab','customlab'])
file_past_hist = pick(all_files, exact='pastHistory.csv.gz', include=['pasthistory'])

print("Tables:")
print("  patient        :", file_patient)
print("  diagnosis      :", file_diagnosis)
print("  apache results :", file_apache_pt)
print("  vitalPeriodic  :", file_vital_per)
print("  vitalAperiodic :", file_vital_aper)
print("  lab            :", file_lab)
print("  pastHistory    :", file_past_hist)

assert file_patient and file_diagnosis, "Missing required tables (patient/diagnosis)."

In [ ]:
db_path = "/content/drive/MyDrive/2025_PatientLevelDataAnalysis/eicu_csvs/eicu_rebuilt.sqlite"
conn = sqlite3.connect(db_path)

def load_to_sql(csv_file, table_name, conn, chunksize=100000):
    full_path = os.path.join("/content/drive/MyDrive/2025_PatientLevelDataAnalysis/eicu_csvs/", csv_file)
    for chunk in pd.read_csv(full_path, compression="gzip", chunksize=chunksize, low_memory=False):
        chunk.to_sql(table_name, conn, if_exists="append", index=False)

def ensure_table(csv_file, table_name):
    if csv_file is None:
        return
    check = conn.execute(f"SELECT name FROM sqlite_master WHERE type='table' AND name='{table_name}'").fetchone()
    if not check:
        print(f"Loading {table_name} ...")
        load_to_sql(csv_file, table_name, conn)
    else:
        print(f"Table {table_name} already exists")

ensure_table(file_patient, "patient")
ensure_table(file_diagnosis, "diagnosis")
ensure_table(file_apache_pt, "apachePatientResult")
ensure_table(file_vital_per, "vitalPeriodic")
ensure_table(file_vital_aper, "vitalAperiodic")
ensure_table(file_lab, "lab")
ensure_table(file_past_hist, "pastHistory")

In [ ]:
# Patient
patient = pd.read_sql_query("SELECT * FROM patient", conn)

# Apache
apache = pd.read_sql_query("SELECT * FROM apachePatientResult", conn)

# Vital periodic
vitals_periodic = pd.read_sql_query("SELECT * FROM vitalPeriodic", conn)
vitals_periodic_agg = vitals_periodic.groupby("patientunitstayid").agg({
    "heartrate": ["mean", "max", "min"],
    "temperature": ["mean", "max", "min"],
    "respiration": ["mean", "max", "min"],
    "sao2": ["mean", "min"]
}).reset_index()
vitals_periodic_agg.columns = ["_".join(col).strip("_") for col in vitals_periodic_agg.columns]

# Vital aperiodic
vitals_aperiodic = pd.read_sql_query("SELECT * FROM vitalAperiodic", conn)
vitals_aperiodic_agg = vitals_aperiodic.groupby("patientunitstayid").agg({
    "noninvasivesystolic": ["mean", "max"],
    "noninvasivediastolic": ["mean", "min"],
    "noninvasivemean": ["mean"],
    "paop": ["mean"]
}).reset_index()
vitals_aperiodic_agg.columns = ["_".join(col).strip("_") for col in vitals_aperiodic_agg.columns]

# Labs
lab = pd.read_sql_query("SELECT patientunitstayid, labname, labresult FROM lab", conn)
lab_agg = lab.groupby(["patientunitstayid", "labname"]).agg({"labresult": "mean"}).unstack()
lab_agg.columns = [f"{lab}_{agg}" for lab, agg in lab_agg.columns]
lab_agg = lab_agg.reset_index()

# Final merged dataset
df = (patient
      .merge(apache, on="patientunitstayid", how="left")
      .merge(vitals_periodic_agg, on="patientunitstayid", how="left")
      .merge(vitals_aperiodic_agg, on="patientunitstayid", how="left")
      .merge(lab_agg, on="patientunitstayid", how="left"))

print(df.shape)
df.head()

In [ ]:
import os
import pandas as pd
import sqlite3

BASE_DIR = "/content/drive/MyDrive/2025_PatientLevelDataAnalysis/eicu_csvs"

def pick(files, exact=None, include=None, exclude=None):
    files_l = [f.lower() for f in files]
    mapping = dict(zip(files_l, files))
    if exact and exact.lower() in mapping:
        return mapping[exact.lower()]
    cand = []
    for fl, orig in mapping.items():
        if include and not any(inc in fl for inc in include):
            continue
        if exclude and any(exc in fl for exc in exclude):
            continue
        cand.append(orig)
    return cand[0] if cand else None

def read_csv_any(path, nrows=2000):
    if path is None: return None
    return pd.read_csv(path, nrows=nrows)

def clean(df):
    if df is None: return None
    df.columns = [c.strip().lower().replace(" ", "_") for c in df.columns]
    return df

KEY_CAND = ['patientunitstayid','admissionid','stay_id','hadm_id','encounterid']
def find_key(cols):
    for k in KEY_CAND:
        if k in cols: return k
    return None

def find_offset_col(df):
    if df is None: return None
    cands = [c for c in df.columns if 'offset' in c]
    pref = ['observationoffset','labresultoffset','unitadmitoffset','icuadmitoffset','hospitaladmitoffset']
    for p in pref:
        if p in cands: return p
    return cands[0] if cands else None

pf  = os.path.join(BASE_DIR, file_patient)
df  = os.path.join(BASE_DIR, file_diagnosis)
apf = os.path.join(BASE_DIR, file_apache_pt) if file_apache_pt else None
vpf = os.path.join(BASE_DIR, file_vital_per) if file_vital_per else None
vaf = os.path.join(BASE_DIR, file_vital_aper) if file_vital_aper else None
lf  = os.path.join(BASE_DIR, file_lab) if file_lab else None
phf = os.path.join(BASE_DIR, file_past_hist) if file_past_hist else None

patients_s   = clean(read_csv_any(pf))
diagnoses_s  = clean(read_csv_any(df))
apache_s     = clean(read_csv_any(apf))
vitalp_s     = clean(read_csv_any(vpf))
vitala_s     = clean(read_csv_any(vaf))
lab_s        = clean(read_csv_any(lf))
pasthist_s   = clean(read_csv_any(phf))

KEY = find_key(set(patients_s.columns))
assert KEY and KEY in diagnoses_s.columns, "Join key not found in patient/diagnosis."

PATIENT_ADMIT_OFFSET = find_offset_col(patients_s)
print("Detected join key:", KEY)
print("Patient admit offset column:", PATIENT_ADMIT_OFFSET)

db_path = os.path.join(BASE_DIR, "eicu_rebuilt.sqlite")
conn = sqlite3.connect(db_path)

patient = pd.read_sql_query("SELECT * FROM patient", conn)
apache  = pd.read_sql_query("SELECT * FROM apachePatientResult", conn)
vitalp  = pd.read_sql_query("SELECT * FROM vitalPeriodic", conn)
vitala  = pd.read_sql_query("SELECT * FROM vitalAperiodic", conn)
lab     = pd.read_sql_query("SELECT patientunitstayid, labname, labresult FROM lab", conn)

vitalp_agg = vitalp.groupby("patientunitstayid").agg({
    "heartrate": ["mean", "max", "min"],
    "temperature": ["mean", "max", "min"],
    "respiration": ["mean", "max", "min"],
    "sao2": ["mean", "min"]
}).reset_index()
vitalp_agg.columns = ["_".join(col).strip("_") for col in vitalp_agg.columns]

vitala_agg = vitala.groupby("patientunitstayid").agg({
    "noninvasivesystolic": ["mean", "max"],
    "noninvasivediastolic": ["mean", "min"],
    "noninvasivemean": ["mean"],
    "paop": ["mean"]
}).reset_index()
vitala_agg.columns = ["_".join(col).strip("_") for col in vitala_agg.columns]

lab_agg = lab.groupby(["patientunitstayid", "labname"]).agg({"labresult": "mean"}).unstack()
lab_agg.columns = [f"{lab}_{agg}" for lab, agg in lab_agg.columns]
lab_agg = lab_agg.reset_index()

df = (patient
      .merge(apache, on="patientunitstayid", how="left")
      .merge(vitalp_agg, on="patientunitstayid", how="left")
      .merge(vitala_agg, on="patientunitstayid", how="left")
      .merge(lab_agg, on="patientunitstayid", how="left"))

print(df.shape)
df.head()

In [ ]:
def load_and_prepare(pf, df, apf=None, vpf=None, vaf=None, lf=None, phf=None, sample_n=2000):
    """Load eICU tables: first a sample for inspection, then the full dataset."""

    def clean(df):
        df.columns = [c.strip().lower().replace(" ", "_") for c in df.columns]
        return df

    patients_s   = read_csv_any(pf, nrows=sample_n)
    diagnoses_s  = read_csv_any(df, nrows=sample_n)
    apache_s     = read_csv_any(apf, nrows=sample_n) if apf else None
    vitalp_s     = read_csv_any(vpf, nrows=sample_n) if vpf else None
    vitala_s     = read_csv_any(vaf, nrows=sample_n) if vaf else None
    lab_s        = read_csv_any(lf,  nrows=sample_n) if lf else None
    pasthist_s   = read_csv_any(phf, nrows=sample_n) if phf else None

    patients_s, diagnoses_s = clean(patients_s), clean(diagnoses_s)
    if apache_s is not None: apache_s = clean(apache_s)
    if vitalp_s is not None: vitalp_s = clean(vitalp_s)
    if vitala_s is not None: vitala_s = clean(vitala_s)
    if lab_s is not None: lab_s = clean(lab_s)
    if pasthist_s is not None: pasthist_s = clean(pasthist_s)

    KEY_CAND = ['patientunitstayid','admissionid','stay_id','hadm_id','encounterid']
    def find_key(cols):
        for k in KEY_CAND:
            if k in cols: return k
        return None

    KEY = find_key(set(patients_s.columns))
    assert KEY and KEY in diagnoses_s.columns, "Join key not found in patient/diagnosis."

    def find_offset_col(df):
        if df is None: return None
        cands = [c for c in df.columns if 'offset' in c]
        pref = ['observationoffset','labresultoffset','unitadmitoffset','icuadmitoffset','hospitaladmitoffset']
        for p in pref:
            if p in cands: return p
        return cands[0] if cands else None

    PATIENT_ADMIT_OFFSET = find_offset_col(patients_s)
    print("Detected key:", KEY, "| patient admit offset column:", PATIENT_ADMIT_OFFSET)

    patients   = clean(read_csv_any(pf))
    diagnoses  = clean(read_csv_any(df))
    apache_pt  = clean(read_csv_any(apf)) if apf else None
    vital_per  = clean(read_csv_any(vpf)) if vpf else None
    vital_aper = clean(read_csv_any(vaf)) if vaf else None
    lab        = clean(read_csv_any(lf))  if lf else None
    past_hist  = clean(read_csv_any(phf)) if phf else None

    print("Shapes -> patients:", patients.shape, "diagnoses:", diagnoses.shape)

    return {
        "patients": patients,
        "diagnoses": diagnoses,
        "apache_pt": apache_pt,
        "vital_per": vital_per,
        "vital_aper": vital_aper,
        "lab": lab,
        "past_hist": past_hist,
        "key": KEY,
        "patient_admit_offset": PATIENT_ADMIT_OFFSET
    }

In [ ]:
import os
import pandas as pd

# uses an existing SQLite connection named `conn`

def read_csv_any(path_or_df, nrows=None):
    """Accepts a file path (csv/tsv/csv.gz) OR a pandas DataFrame."""
    if path_or_df is None:
        return None
    if isinstance(path_or_df, pd.DataFrame):
        return path_or_df if nrows is None else path_or_df.head(nrows)
    path = path_or_df
    compression = 'gzip' if str(path).endswith('.gz') else 'infer'
    sep = '\t' if str(path).endswith('.tsv') else ','
    return pd.read_csv(path, sep=sep, compression=compression, nrows=nrows)

def clean(df):
    df.columns = [c.strip().lower().replace(" ", "_") for c in df.columns]
    return df

def load_to_sql(df, name):
    if df is not None:
        df.to_sql(name, conn, if_exists="replace", index=False)
        print(f"Loaded table: {name} ({df.shape[0]} rows)")
    return df

def load_and_prepare(pf, diagf, apf=None, vpf=None, vaf=None, lf=None, phf=None, nrows=None):
    """pf/diagf/... should be FILE PATHS (or DataFrames)."""
    patients   = load_to_sql(clean(read_csv_any(pf,   nrows=nrows)), "patients")
    diagnoses  = load_to_sql(clean(read_csv_any(diagf,nrows=nrows)), "diagnoses")
    apache_pt  = load_to_sql(clean(read_csv_any(apf,  nrows=nrows)), "apache_pt")   if apf else None
    vital_per  = load_to_sql(clean(read_csv_any(vpf,  nrows=nrows)), "vital_per")   if vpf else None
    vital_aper = load_to_sql(clean(read_csv_any(vaf,  nrows=nrows)), "vital_aper")  if vaf else None
    lab        = load_to_sql(clean(read_csv_any(lf,   nrows=nrows)), "lab")         if lf else None
    past_hist  = load_to_sql(clean(read_csv_any(phf,  nrows=nrows)), "past_hist")   if phf else None
    return {
        "patients": patients, "diagnoses": diagnoses, "apache_pt": apache_pt,
        "vital_per": vital_per, "vital_aper": vital_aper, "lab": lab,
        "past_hist": past_hist, "key": "patientunitstayid"
    }

    pf     = os.path.join(BASE_DIR, file_patient)
diagf  = os.path.join(BASE_DIR, file_diagnosis)   # <-- use diagf, NOT a DataFrame
apf    = os.path.join(BASE_DIR, file_apache_pt) if file_apache_pt else None
vpf    = os.path.join(BASE_DIR, file_vital_per)  if file_vital_per else None
vaf    = os.path.join(BASE_DIR, file_vital_aper) if file_vital_aper else None
lf     = os.path.join(BASE_DIR, file_lab)        if file_lab else None
phf    = os.path.join(BASE_DIR, file_past_hist)  if file_past_hist else None

tables = load_and_prepare(pf, diagf, apf, vpf, vaf, lf, phf)

pd.read_sql("SELECT patientunitstayid, gender, age FROM patients LIMIT 5", conn)

In [ ]:
import sqlite3, pandas as pd

conn = sqlite3.connect("your_database.db")

patients = pd.read_sql("SELECT * FROM patients", conn)
diagnoses = pd.read_sql("SELECT * FROM diagnoses", conn)

print("Patients shape:", patients.shape)
print("Diagnoses shape:", diagnoses.shape)
print(diagnoses.head())

diagnoses_expanded = (
    diagnoses.assign(icd9code=diagnoses['icd9code'].str.split(','))
             .explode('icd9code')
             .reset_index(drop=True)
)

diagnoses_expanded['icd9code'] = (
    diagnoses_expanded['icd9code'].str.strip()
)

print("Original shape:", diagnoses.shape)
print("Expanded shape:", diagnoses_expanded.shape)
print(diagnoses_expanded.head(10))

In [ ]:
import os
import pandas as pd

OUT = os.path.join(BASE_DIR, 'outputs')
os.makedirs(OUT, exist_ok=True)

KEY = "patientunitstayid"

diagnoses_expanded = (
    diagnoses.assign(icd_code=diagnoses['icd9code'].str.split(','))
             .explode('icd_code')
             .reset_index(drop=True)
)

diagnoses_expanded['icd_code'] = (
    diagnoses_expanded['icd_code']
    .astype(str)
    .str.strip()
    .str.replace('.', '', regex=False)
    .str.upper()
)

print("Original shape:", diagnoses.shape)
print("Expanded shape:", diagnoses_expanded.shape)
print(diagnoses_expanded.head(10))

dx_counts = (
    diagnoses_expanded.groupby("icd_code")[KEY].nunique()
    .sort_values(ascending=False)
    .reset_index()
    .rename(columns={KEY: "unique_stays"})
)
dx_counts.to_csv(os.path.join(OUT, "diagnosis_frequency.csv"), index=False)

CURATED_GROUPS = {
    'sepsis':        ['A41','038','R652','9959'],
    'pneumonia':     ['J18','486'],
    'ami':           ['I21','410'],
    'aki':           ['N17','584'],
    'chf':           ['I50','428'],
    'copd':          ['J44','496'],
    'stroke':        ['I63','I61','434','431'],
    'resp_failure':  ['J96','51881'],
    'gi_bleed':      ['K922','578'],
    'dka':           ['E101','E111','2501'],
    'shock':         ['R57','7855'],
    'uti':           ['N390','5990'],
    'ards':          ['J80','51882'],
}

def code_to_curated_group(code):
    for grp, prefs in CURATED_GROUPS.items():
        if any(code.startswith(p) for p in prefs):
            return grp
    return None

diag_grp = diagnoses_expanded[[KEY, "icd_code"]].copy()
diag_grp["curated_group"] = diag_grp["icd_code"].apply(code_to_curated_group)

diag_grp["prefix3"] = diag_grp["icd_code"].str[:3]

prefix_freq = (
    diag_grp.groupby("prefix3")[KEY].nunique()
    .sort_values(ascending=False)
    .reset_index()
    .rename(columns={KEY: "unique_stays"})
)
prefix_freq.to_csv(os.path.join(OUT, "diagnosis_prefix3_frequency.csv"), index=False)

TOP_PREFIX_K = 15
top_prefixes = set(prefix_freq.head(TOP_PREFIX_K)["prefix3"].tolist())

labels = pd.DataFrame({KEY: patients[KEY].unique()})
labels = labels.sort_values(KEY).reset_index(drop=True)

for grp in CURATED_GROUPS.keys():
    stays = diag_grp.loc[diag_grp["curated_group"] == grp, KEY].unique()
    labels[grp] = labels[KEY].isin(stays).astype(int)

for p in sorted(top_prefixes):
    stays = diag_grp.loc[diag_grp["prefix3"] == p, KEY].unique()
    labels[f"icd_{p}"] = labels[KEY].isin(stays).astype(int)

MIN_POS = 25
label_cols_all = [c for c in labels.columns if c != KEY]
pos_counts = labels[label_cols_all].sum().sort_values(ascending=False)

keep_cols = [c for c in label_cols_all if labels[c].sum() >= MIN_POS]
dropped = sorted(set(label_cols_all) - set(keep_cols))

labels = labels[[KEY] + keep_cols]

print("Label columns kept:", keep_cols)
print("Dropped (too few positives):", dropped)
print("Positives by label:")
print(labels[keep_cols].sum().sort_values(ascending=False))

labels.to_csv(os.path.join(OUT, "labels_multi.csv"), index=False)

In [ ]:
import os
import pandas as pd
import sqlite3

BASE_DIR = "/content/drive/MyDrive/2025_PatientLevelDataAnalysis/eicu_csvs"
db_path = os.path.join(BASE_DIR, "eicu_rebuilt.sqlite")
conn = sqlite3.connect(db_path)

def pick(files, exact=None, include=None, exclude=None):
    files_l = [f.lower() for f in files]
    mapping = dict(zip(files_l, files))
    if exact and exact.lower() in mapping:
        return mapping[exact.lower()]
    cand = []
    for fl, orig in mapping.items():
        if include and not any(inc in fl for inc in include):
            continue
        if exclude and any(exc in fl for exc in exclude):
            continue
        cand.append(orig)
    return cand[0] if cand else None

all_files = os.listdir(BASE_DIR)

file_patient   = pick(all_files, exact='patient.csv.gz', include=['patient'], exclude=['apache'])
file_diagnosis = pick(all_files, exact='diagnosis.csv.gz', include=['diagnosis','admissiondx'])
file_apache_pt = pick(all_files, exact='apachePatientResult.csv.gz', include=['apachepatientresult'])
file_vital_per = pick(all_files, exact='vitalPeriodic.csv.gz', include=['vitalperiodic'])
file_vital_aper= pick(all_files, exact='vitalAperiodic.csv.gz', include=['vitalaperiodic'])
file_lab       = pick(all_files, exact='lab.csv.gz', include=['lab'], exclude=['microlab','customlab'])
file_past_hist = pick(all_files, exact='pastHistory.csv.gz', include=['pasthistory'])

files_to_load = {
    "patients": file_patient,
    "diagnoses": file_diagnosis,
    "apache_pt": file_apache_pt,
    "vital_per": file_vital_per,
    "vital_aper": file_vital_aper,
    "lab": file_lab,
    "past_hist": file_past_hist
}

for table_name, fname in files_to_load.items():
    if fname is None:
        print(f"Skipping {table_name}: file not found")
        continue
    path = os.path.join(BASE_DIR, fname)
    print(f"Loading {table_name} from {fname}...")
    df = pd.read_csv(path, compression='gzip' if fname.endswith('.gz') else None)
    df.columns = [c.strip().lower().replace(" ", "_") for c in df.columns]  # normalize columns
    df.to_sql(table_name, conn, if_exists="replace", index=False)
    print(f"Loaded table: {table_name} ({df.shape[0]} rows, {df.shape[1]} cols)")

conn.close()
print("All tables loaded into SQLite!")

import sqlite3
import pandas as pd
import os

BASE_DIR = "/content/drive/MyDrive/2025_PatientLevelDataAnalysis/eicu_csvs"
db_path = os.path.join(BASE_DIR, "eicu_rebuilt.sqlite")
OUT = os.path.join(BASE_DIR, "outputs")
os.makedirs(OUT, exist_ok=True)

conn = sqlite3.connect(db_path)

patients = pd.read_sql_query("SELECT * FROM patients", conn)
diagnoses = pd.read_sql_query("SELECT * FROM diagnoses", conn)
apache_pt = pd.read_sql_query("SELECT * FROM apache_pt", conn)
vital_per = pd.read_sql_query("SELECT * FROM vital_per", conn)
vital_aper = pd.read_sql_query("SELECT * FROM vital_aper", conn)
lab = pd.read_sql_query("SELECT * FROM lab", conn)
past_hist = pd.read_sql_query("SELECT * FROM past_hist", conn)

conn.close()

KEY = "patientunitstayid"

feat = patients[[KEY]].copy()
for col in ['age','gender','ethnicity','unittype','unitstaytype','hospitaladmitsource','hospitaldischargestatus','unitdischargestatus']:
    if col in patients.columns:
        if col == 'age':
            feat['age'] = pd.to_numeric(patients['age'], errors='coerce')
        else:
            feat[col] = patients[col].astype(str)

admit_off, disc_off = None, None
for c in ['unitadmitoffset','icuadmitoffset','hospitaladmitoffset']:
    if c in patients.columns: admit_off = c; break
for c in ['unitdischargeoffset','icudischargeoffset','hospitaldischargeoffset']:
    if c in patients.columns: disc_off = c; break

if admit_off and disc_off:
    los_days = (pd.to_numeric(patients[disc_off], errors='coerce') -
                pd.to_numeric(patients[admit_off], errors='coerce'))/(60*24)
    feat['los_days'] = los_days

if apache_pt is not None:
    cols_ap = [c for c in apache_pt.columns if any(x in c for x in ['apache','predict','score','mortality'])]
    if cols_ap:
        ap = apache_pt[[KEY]+cols_ap].copy()
        for c in cols_ap:
            ap[c] = pd.to_numeric(ap[c], errors='ignore')
        feat = feat.merge(ap, on=KEY, how='left')

def aggregate_first_6h(df, key_col, offset_candidates, prefix):
    if df is None: return None
    off = None
    for c in offset_candidates:
        if c in df.columns: off = c; break
    if off is None: return None
    sub = df[(pd.to_numeric(df[off], errors='coerce') >= 0) & (pd.to_numeric(df[off], errors='coerce') <= 360)].copy()
    if sub.empty: return None

    num_cols = [c for c in sub.columns if c not in [key_col, off] and pd.api.types.is_numeric_dtype(sub[c])]
    if not num_cols: return None
    ag = sub.groupby(key_col)[num_cols].agg(['min','max','mean']).reset_index()
    ag.columns = [key_col] + [f"{prefix}__{c}__{stat}" for c in num_cols for stat in ['min','max','mean']]
    return ag

vp_agg = aggregate_first_6h(vital_per, KEY, ['observationoffset','chartoffset','offset'], 'vitalp')
if vp_agg is not None:
    feat = feat.merge(vp_agg, on=KEY, how='left')

va_agg = aggregate_first_6h(vital_aper, KEY, ['observationoffset','chartoffset','offset'], 'vitala')
if va_agg is not None:
    feat = feat.merge(va_agg, on=KEY, how='left')

if lab is not None:
    lb = lab.copy()
    lb_off = next((c for c in ['labresultoffset','chartoffset','observationoffset','offset'] if c in lb.columns), None)
    val_col = next((c for c in ['labresult','labvalue','resultvalue','value'] if c in lb.columns), None)
    lname = 'labname' if 'labname' in lb.columns else None
    if lb_off and val_col and lname:
        lb[val_col] = pd.to_numeric(lb[val_col], errors='coerce')
        common = (lb[lb[val_col].notna()].groupby(lname)[KEY].nunique().sort_values(ascending=False).head(20).index.tolist())
        from functools import reduce
        aggs = []
        for ln in common:
            sub = lb[(lb[lname]==ln) & lb[val_col].notna()]
            ag = aggregate_first_6h(sub, KEY, [lb_off], f"lab__{str(ln).replace(' ','_').lower()}")
            if ag is not None: aggs.append(ag)
        if aggs:
            feat = reduce(lambda L,R: L.merge(R,on=KEY,how='left'), [feat]+aggs)

if past_hist is not None:
    ph = past_hist.copy()
    text_col = next((c for c in ['pasthistorytext','pasthistoryitem','pasthistoryvalue','pasthistorypath'] if c in ph.columns), None)
    if text_col:
        ph[text_col] = ph[text_col].astype(str).str.lower()
        agg_txt = ph.groupby(KEY)[text_col].apply(lambda s: ' | '.join(s.astype(str))).reset_index()
        flags = {
            'hx_chf': ['heart failure','congestive heart','chf'],
            'hx_copd': ['copd','chronic obstructive'],
            'hx_ckd': ['ckd','chronic kidney','renal failure'],
            'hx_dm':  ['diabetes','dm '],
            'hx_cancer': ['cancer','malignancy','carcinoma','tumor'],
        }
        for col, kws in flags.items():
            agg_txt[col] = agg_txt[text_col].apply(lambda t: any(k in t for k in kws))
        feat = feat.merge(agg_txt.drop(columns=[text_col]), on=KEY, how='left')

feat.to_csv(os.path.join(OUT, 'features_6h.csv'), index=False)
print("6-hour feature matrix shape:", feat.shape)

In [ ]:
import os
import pandas as pd
import sqlite3
from functools import reduce

BASE_DIR = "/content/drive/MyDrive/2025_PatientLevelDataAnalysis/eicu_csvs"
db_path = os.path.join(BASE_DIR, "eicu_rebuilt.sqlite")
OUT = os.path.join(BASE_DIR, "outputs")
os.makedirs(OUT, exist_ok=True)

conn = sqlite3.connect(db_path)

patients   = pd.read_sql_query("SELECT * FROM patients", conn)
diagnoses  = pd.read_sql_query("SELECT * FROM diagnoses", conn)
apache_pt  = pd.read_sql_query("SELECT * FROM apache_pt", conn)
vital_per  = pd.read_sql_query("SELECT * FROM vital_per", conn)
vital_aper = pd.read_sql_query("SELECT * FROM vital_aper", conn)
lab        = pd.read_sql_query("SELECT * FROM lab", conn)
past_hist  = pd.read_sql_query("SELECT * FROM past_hist", conn)

conn.close()

KEY = "patientunitstayid"

feat = patients[[KEY]].copy()

for col in ['age','gender','ethnicity','unittype','unitstaytype','hospitaladmitsource',
            'hospitaldischargestatus','unitdischargestatus']:
    if col in patients.columns:
        if col == 'age':
            feat['age'] = pd.to_numeric(patients['age'], errors='coerce')
        else:
            feat[col] = patients[col].astype(str)

admit_off = next((c for c in ['unitadmitoffset','icuadmitoffset','hospitaladmitoffset'] if c in patients.columns), None)
disc_off  = next((c for c in ['unitdischargeoffset','icudischargeoffset','hospitaldischargeoffset'] if c in patients.columns), None)
if admit_off and disc_off:
    feat['los_days'] = (pd.to_numeric(patients[disc_off], errors='coerce') -
                        pd.to_numeric(patients[admit_off], errors='coerce')) / (60*24)

if apache_pt is not None:
    cols_ap = [c for c in apache_pt.columns if any(x in c for x in ['apache','predict','score','mortality'])]
    if cols_ap:
        ap = apache_pt[[KEY]+cols_ap].copy()
        for c in cols_ap:
            ap[c] = pd.to_numeric(ap[c], errors='ignore')
        feat = feat.merge(ap, on=KEY, how='left')

def aggregate_first_6h(df, key_col, offset_candidates, prefix):
    if df is None: return None
    off = next((c for c in offset_candidates if c in df.columns), None)
    if off is None: return None
    sub = df[(pd.to_numeric(df[off], errors='coerce') >= 0) &
             (pd.to_numeric(df[off], errors='coerce') <= 360)].copy()
    if sub.empty: return None
    num_cols = [c for c in sub.columns if c not in [key_col, off] and pd.api.types.is_numeric_dtype(sub[c])]
    if not num_cols: return None
    ag = sub.groupby(key_col)[num_cols].agg(['min','max','mean']).reset_index()
    ag.columns = [key_col] + [f"{prefix}__{c}__{stat}" for c in num_cols for stat in ['min','max','mean']]
    return ag

vp_agg = aggregate_first_6h(vital_per, KEY, ['observationoffset','chartoffset','offset'], 'vitalp')
if vp_agg is not None: feat = feat.merge(vp_agg, on=KEY, how='left')

va_agg = aggregate_first_6h(vital_aper, KEY, ['observationoffset','chartoffset','offset'], 'vitala')
if va_agg is not None: feat = feat.merge(va_agg, on=KEY, how='left')

if lab is not None:
    lb = lab.copy()
    lb_off  = next((c for c in ['labresultoffset','chartoffset','observationoffset','offset'] if c in lb.columns), None)
    val_col = next((c for c in ['labresult','labvalue','resultvalue','value'] if c in lb.columns), None)
    lname   = 'labname' if 'labname' in lb.columns else None
    if lb_off and val_col and lname:
        lb[val_col] = pd.to_numeric(lb[val_col], errors='coerce')
        common = lb[lb[val_col].notna()].groupby(lname)[KEY].nunique().sort_values(ascending=False).head(20).index.tolist()
        aggs = []
        for ln in common:
            sub = lb[(lb[lname]==ln) & lb[val_col].notna()]
            ag = aggregate_first_6h(sub, KEY, [lb_off], f"lab__{str(ln).replace(' ','_').lower()}")
            if ag is not None: aggs.append(ag)
        if aggs:
            feat = reduce(lambda L,R: L.merge(R,on=KEY,how='left'), [feat]+aggs)

if past_hist is not None:
    ph = past_hist.copy()
    text_col = next((c for c in ['pasthistorytext','pasthistoryitem','pasthistoryvalue','pasthistorypath'] if c in ph.columns), None)
    if text_col:
        ph[text_col] = ph[text_col].astype(str).str.lower()
        agg_txt = ph.groupby(KEY)[text_col].apply(lambda s: ' | '.join(s.astype(str))).reset_index()
        flags = {
            'hx_chf': ['heart failure','congestive heart','chf'],
            'hx_copd': ['copd','chronic obstructive'],
            'hx_ckd': ['ckd','chronic kidney','renal failure'],
            'hx_dm':  ['diabetes','dm '],
            'hx_cancer': ['cancer','malignancy','carcinoma','tumor'],
        }
        for col, kws in flags.items():
            agg_txt[col] = agg_txt[text_col].apply(lambda t: any(k in t for k in kws))
        feat = feat.merge(agg_txt.drop(columns=[text_col]), on=KEY, how='left')


code_col = 'icd9code' if 'icd9code' in diagnoses.columns else ('icd10code' if 'icd10code' in diagnoses.columns else None)
diagnoses[code_col] = diagnoses[code_col].astype(str).str.replace('.','', regex=False).str.upper()

CURATED_GROUPS = {
    'sepsis': ['A41','038','R652','9959'],

}

def code_to_curated_group(code):
    for grp, prefs in CURATED_GROUPS.items():
        if any(code.startswith(p) for p in prefs):
            return grp
    return None

diag_grp = diagnoses[[KEY, code_col]].copy()
diag_grp['curated_group'] = diag_grp[code_col].apply(code_to_curated_group)

labels = pd.DataFrame({KEY: patients[KEY].unique()})
for grp in CURATED_GROUPS.keys():
    stays = diag_grp.loc[diag_grp['curated_group']==grp, KEY].unique()
    labels[grp] = labels[KEY].isin(stays).astype(int)

df_all = labels.merge(feat, on=KEY, how="inner")
print("Final dataset shape:", df_all.shape)
df_all.to_csv(os.path.join(OUT,"df_all_6h.csv"), index=False)

In [ ]:
import os
import json
import numpy as np
import pandas as pd
import xgboost as xgb
import shap
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split, StratifiedKFold, RandomizedSearchCV
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.metrics import (roc_auc_score, average_precision_score, precision_recall_curve,
                             f1_score, confusion_matrix, classification_report)

OUT = os.path.join(BASE_DIR, "outputs")
os.makedirs(OUT, exist_ok=True)

df_all = labels.merge(feat, on=KEY, how='inner')

label_cols = [c for c in labels.columns if c != KEY]
cat_cols = [c for c in df_all.columns if df_all[c].dtype == 'object' and c not in [KEY] + label_cols]
num_cols = [c for c in df_all.columns if c not in cat_cols + [KEY] + label_cols]

print("Total samples:", len(df_all))
print("Num features:", len(num_cols), "| Cat features:", len(cat_cols), "| Labels:", len(label_cols))

pre = ColumnTransformer([
    ("num", SimpleImputer(strategy="median"), num_cols),
    ("cat", Pipeline([("imp", SimpleImputer(strategy="most_frequent")),
                      ("oh", OneHotEncoder(handle_unknown="ignore"))]), cat_cols)
])

tree_method = "hist"

RESULTS = []
TOP_SHAP_LABELS = 5
pos_by_label = {lbl: int(df_all[lbl].sum()) for lbl in label_cols}
top_for_shap = [lbl for lbl, _ in sorted(pos_by_label.items(), key=lambda x: x[1], reverse=True)[:TOP_SHAP_LABELS]]

for lbl in label_cols:
    print("\n============================")
    print(f"Training for label: {lbl} | positives: {pos_by_label[lbl]}")
    y = df_all[lbl].astype(int)
    if y.sum() < 25:
        print("Skipping label due to low positives (<25).")
        continue

    X = df_all[num_cols + cat_cols]
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=0.2, stratify=y, random_state=42
    )

    neg = (y_train == 0).sum()
    pos = (y_train == 1).sum()
    spw = max(1.0, neg / max(1, pos))

    clf = xgb.XGBClassifier(
        objective="binary:logistic",
        tree_method=tree_method,
        eval_metric="logloss",
        random_state=42,
        scale_pos_weight=spw
    )

    pipe = Pipeline([("prep", pre), ("model", clf)])

    param_distributions = {
        "model__n_estimators": [300, 500, 700],
        "model__max_depth": [3, 4, 5, 6],
        "model__learning_rate": [0.02, 0.05, 0.1],
        "model__subsample": [0.6, 0.8, 1.0],
        "model__colsample_bytree": [0.6, 0.8, 1.0],
        "model__min_child_weight": [1, 2, 5, 10],
    }

    cv = StratifiedKFold(n_splits=3, shuffle=True, random_state=42)
    search = RandomizedSearchCV(pipe, param_distributions, n_iter=12, cv=cv,
                                scoring="roc_auc", n_jobs=-1, verbose=1, random_state=42)
    search.fit(X_train, y_train)
    best_pipe = search.best_estimator_

    y_proba = best_pipe.predict_proba(X_test)[:,1]
    roc_auc = roc_auc_score(y_test, y_proba)
    pr_auc  = average_precision_score(y_test, y_proba)

    prec, rec, th = precision_recall_curve(y_test, y_proba)
    f1 = (2*prec*rec)/(prec+rec+1e-12)
    ix = int(np.nanargmax(f1))
    best_th = th[ix] if ix < len(th) else 0.5

    y_pred = (y_proba >= best_th).astype(int)
    cm = confusion_matrix(y_test, y_pred)
    report = classification_report(y_test, y_pred, digits=3, output_dict=True)

    label_dir = os.path.join(OUT, f"label_{lbl}")
    os.makedirs(label_dir, exist_ok=True)

    with open(os.path.join(label_dir, "metrics.json"), "w") as f:
        json.dump({
            "label": lbl, "positives": int(y.sum()),
            "roc_auc": float(roc_auc), "pr_auc": float(pr_auc),
            "threshold": float(best_th), "confusion_matrix": cm.tolist(),
            "classification_report": report
        }, f, indent=2)

    if lbl.lower() == "sepsis":
      import seaborn as sns

      ax_labels = ["0", "1"]
      plt.figure(figsize=(5,4))
      sns.heatmap(cm, annot=True, fmt='d', cmap="RdYlGn_r", cbar=False,
                xticklabels=ax_labels, yticklabels=ax_labels)

      plt.text(0.5, 0.5, 'TP', fontsize=14, ha='center', va='center', color='black')
      plt.text(1.5, 0.5, 'FN', fontsize=14, ha='center', va='center', color='black')
      plt.text(0.5, 1.5, 'FP', fontsize=14, ha='center', va='center', color='black')
      plt.text(1.5, 1.5, 'TN', fontsize=14, ha='center', va='center', color='black')

      plt.xlabel('Predicted')
      plt.ylabel('Actual')
      plt.title(f'Confusion Matrix for {lbl}')
      plt.tight_layout()
      plt.savefig(os.path.join(label_dir, "confusion_matrix.png"), dpi=200, bbox_inches='tight')
      plt.close()

    import joblib
    joblib.dump(best_pipe, os.path.join(label_dir, "model.pkl"))

    if lbl in top_for_shap:
        prep_step = best_pipe.named_steps['prep']
        model_step = best_pipe.named_steps['model']
        X_train_t = prep_step.transform(X_train)
        num_names = num_cols
        cat_oh = prep_step.transformers_[1][1].named_steps['oh'].get_feature_names_out(cat_cols) if cat_cols else []
        feature_names = list(num_names) + list(cat_oh)

        explainer = shap.TreeExplainer(model_step)
        nsample = min(4000, X_train_t.shape[0])
        sv = explainer.shap_values(X_train_t[:nsample])

        plt.figure()
        shap.summary_plot(sv, X_train_t[:nsample], feature_names=feature_names, show=False)
        plt.tight_layout()
        plt.savefig(os.path.join(label_dir, "shap_summary.png"), dpi=200, bbox_inches='tight')
        plt.close()

    RESULTS.append({
        "label": lbl,
        "positives": int(y.sum()),
        "roc_auc": float(roc_auc),
        "pr_auc": float(pr_auc),
        "threshold": float(best_th),
        "f1_at_threshold": float(f1_score(y_test, y_pred)),
        "tn": int(cm[0,0]), "fp": int(cm[0,1]), "fn": int(cm[1,0]), "tp": int(cm[1,1])
    })

res_df = pd.DataFrame(RESULTS).sort_values(["positives","pr_auc"], ascending=[False, False])
summary_path = os.path.join(OUT, "multilabel_results_summary.csv")
res_df.to_csv(summary_path, index=False)
print(f"\nSaved per-label results to {summary_path}")
display(res_df.head(10))